<a href="https://colab.research.google.com/github/kirawc/semantic-distance/blob/main/createStimuliLists.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# set up; import; define

In [ ]:
# Import everything
import os, json
import pandas as pd
import random
import copy
import sqlite3
import sys
import numpy as np
import csv  


In [ ]:
# Define things
stimDict = {"animal":["a1", "a2", "a3","a4","a5","a6","a7","a8","a9","a10"],
            "instruments":["i1", "i2", "i3","i4","i5","i6","i7","i8","i9","i10"],
            "household":["h1", "h2", "h3","h4","h5","h6","h7","h8","h9","h10"]};

nRates = 20; # total number of ratings you want for each trio 
subsPerSet = 6; # how many subs does it take to get a full rating set

# Put it all together

In [ ]:
# Run functions
trioDict = makeTrios(stimDict) # takes original dictionary and creates dictionary of all the possible (within category) trios
print(len(trioDict))
sub = 0;

# Randomize 
for i in range(nRates): # how many SETS of ratings you want

  cSet = makeSplits(trioDict, subsPerSet);

  for j in range(subsPerSet): # how many SPLITS (subs) you want per SET of ratings
    cSplit = cSet[j];
    df = pd.DataFrame(columns=['sub','set','split','modality','prompt','opt1','opt2'])
    sub += 1;

    for k in range(len(cSplit)):
      cLine = cSplit[k];
      df.loc[k] = [sub, i, j, cLine[0], cLine[1], cLine[2][0], cLine[2][1]];

    #with open('/content/drive/MyDrive/*research/semantic distance/randomizations/SEMDIST_sub'+str(sub)+'_set'+str(i)+'_split'+str(j)+'_9Feb.csv', 'w') as f:
      #df.to_csv(f)
    print("Sub:", sub, " Length:", len(df))

#######
#data = {'sub':[1,2,3], 'set':[1,1,1], 'split':[1,1,1], 'modality':['v','a','v'], 'prompt': ['a1','a1','a1'], 'opt1':['a1','a2','a3'], 'opt2':['a3','a3','a3']}
#df = pd.DataFrame(data)
#print(df)

In [ ]:
#cSet = makeSplits(trioDict, subsPerSet);
print(type(cSet))

In [ ]:
total = 2200;
nSplit = 6;
trialsPer = 2200/nSplit;
print(trialsPer)
secPerTrial = 3;
totalTime = (trialsPer * secPerTrial)/60;
print(totalTime)

# create combinations


In [ ]:
def makeTrios(dict):

  """
  Expands original dict 
  Each stim is now a dict with lists of each possible pair of options, excluding repeats

  structure -- 
  {CATEGORY1: {STIM1:[pair1A, pair1B], STIM2:[pair2A,pair2B]}, CATEGORY2: .... }
  """
  
  resdict = {} # the final 
  categories = list(dict.keys())

  for cat in categories:  # Loop through each category

    stimlist = list(dict[cat]) # get category
    promptDict = {}
    
    for prompt in stimlist: # Loop through all prompts

      everyPairForThisPrompt = []; # reset list of pairs for this prompt  
      optionList1 = [x for x in stimlist if x != prompt]

      # add in matched pair as a check
      controlpair = (prompt, random.choice(optionList1))
      everyPairForThisPrompt.append(controlpair)

      for i in range(len(optionList1)): # Loop through all but current prompt

          opt1 = optionList1[i];
          optionList2 = [x for x in optionList1 if x != opt1]

          for j in range(len(optionList2)): # Loop through all but prompt & opt1

            opt2 = optionList2[j];
            cpair = (opt1, opt2)
            everyPairForThisPrompt.append(cpair)
      
      pairs = removeDuplicates(everyPairForThisPrompt);
      promptDict[prompt] = pairs; 
    
      resdict[cat] = promptDict; 

  return resdict

In [ ]:
def removeDuplicates(lst): 
      
  res = list(set(tuple(sorted(sub)) for sub in lst)) 

  return res

# create randomized splits

In [ ]:
def makeSplits(dict, nSplit):

  """
  (Psuedo) Randomize which trials [prompt & option pairs] a participant will see
  1 participant = 1 split
  ? splits = 1 set

  Pseudo random bc even nums of category & prompt 
  So every participant has the same ex posure to each category & each prompt
  """

  # INPUT STRUCTURE
  # {category: {prompt1:[opt1A,op2A],....}, {prompt2: [opt1B, opt2B],....}}
  allDict = dict;
  allList = list(allDict.keys()) # dictionary with everycategory 

  # OUTPUT STRUCTURE
  # res = [  [ [a, a1,a2,a3], [v, b1,b2,b3] ], [ ]]
  # list of splits with trios
  res = [[] for i in range(nSplit)]
  resSplits = [[] for i in range(nSplit)]
  modalities = ["a","v"]

  for cat in allDict: # loop through outer categories

    catDict = allDict[cat]; # dictinary with current category
    prompts = list(allDict[cat].keys()) # keys for current dictionary

    for modality in modalities:
      
      for prompt in prompts:

        optpairs = list(catDict[prompt]) # options for this prompt for this category 
        trios = [[modality, prompt] + [lst] for lst in optpairs]
        shuffledTrios = random.sample(trios,len(trios)) # shuffle up the options
        #splitList = split_list(shuffledTrios,nSplit)
        splitList = splitter(shuffledTrios,nSplit)

        for n in range(nSplit):
          resSplits[n].extend(splitList[n])
  
  for n in range(nSplit):
    res[n] = random.sample(resSplits[n], len(resSplits[n]))

  return res

In [ ]:
makeSplits(trioDict,6)

NameError: ignored

In [ ]:
def splitter(lst,n):
  return [lst[i::n] for i in xrange(n)]  

In [ ]:
def split_list(alist, wanted_parts):
    length = len(alist)
    return [ alist[i*length // wanted_parts: (i+1)*length // wanted_parts] 
             for i in range(wanted_parts) ]

## Junk

In [ ]:
##### JUNK

  categoryList = list(dict.keys())

  # loop through each category
  for cat in categories: 
    promptDict = dict[cat];
    prompts = promptDict.keys();

    for prompt in prompts:
      
      ckey = promptDict[prompt];
      optionPairs = promptDict[ckey]; # every 
      nEach = len(list(optPairs))/nSplit; # 
      shuffledPairs = random.sample(optionPairs,len(optionPairs)) # shuffle the full list
      print(shuffledPairs)
      counter = 0;      


dict = trioDict;
categoryList = list(dict.keys())

nSplit = 2;

promptDict = trioDict[categoryList[0]]
prompts = list(promptDict.keys());
optionPairs = list(promptDict[prompts[0]])
nPairs = len(optionPairs)

#print(optionPairs)

shuffledOptPairs = random.sample(optionPairs,nPairs)
shuffledOptPairs2 = random.sample(optionPairs,nPairs)
finalList = []

for i in range(len(shuffledOptPairs)):
  pair = shuffledOptPairs[i]
  trio = ["a3"];
  trio.append(pair)
  finalList.append(trio)

print(finalList)